<a href="https://colab.research.google.com/github/dkisselev-zz/mmc-pipeline/blob/main/Get_Sample_MetaData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
ok# -*- coding: utf-8 -*-
"""
## Notebook: SRA and BioSample Metadata Extraction from Google Sheet

This notebook takes a list of SRA/BioProject accession codes from a specified
Google Sheet, queries the NCBI Entrez E-Utilities API, and extracts detailed
metadata for each entry.

It gathers information about the SRA records (like instrument and library names)
and the associated BioSample records (like host organism).
"""

# ==============================================================================
# STEP 1: SETUP (IMPORTS AND AUTHENTICATION)
# ==============================================================================
import os
import io
import requests
import time
import re
import json
import pandas as pd
from ftplib import FTP
from collections import defaultdict
import xml.etree.ElementTree as ET
from google.colab import auth
from google.auth import default
import google.generativeai as genai
from google.colab import userdata
from google.colab.data_table import DataTable
import gspread

# --- Authenticate to access Google Sheet ---
# This will prompt you to log in and authorize access.
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

print("Authentication successful.")


Authentication successful.


### Step 2

In [ ]:
# ==============================================================================
# STEP 2: LLM NORMALIZER AND CORE NCBI DATA FUNCTION
# ==============================================================================
# --- LLM Class for Host Normalization ---
class LLMNormalizer:
    def __init__(self):
        try:
            API_KEY = userdata.get('GOOGLE_API_KEY')
            genai.configure(api_key=API_KEY)
            self.model = genai.GenerativeModel("gemini-1.5-flash")
            print("LLM Normalizer initialized successfully.")
        except Exception as e:
            print(f"ERROR: Could not initialize LLM. Normalization will be skipped. Details: {e}")
            self.model = None

    def _call_gemini_with_retries(self, prompt, retries=3, delay=5):
        if not self.model: return "LLM Not Initialized"
        for attempt in range(retries):
            try:
                # Request JSON output from the model
                response = self.model.generate_content(prompt, generation_config={"response_mime_type": "application/json"})
                return response.text.strip()
            except Exception as e:
                print(f"  > LLM Error (Attempt {attempt + 1}/{retries}): {e}")
                if attempt < retries - 1:
                    time.sleep(delay)
                else: return "{}" # Return empty JSON on failure

    def create_host_normalization_map(self, host_list):
        if not self.model or not host_list: return {}

        unique_hosts = sorted(list(set(h.strip() for h in host_list if h and h.strip())))

        prompt= (
                """You are a bioinformatics expert tasked with cleaning data. Your job is to analyze the following JSON list of
                   host names and standardize them according to these strict rules.

                ## Core Principle:
                Your primary goal is to identify the **source organism** for each item.

                ## Rules:
                1.  **The Host Extraction Rule:** If an input string names a host organism (e.g., 'human', 'mouse', 'rat', 'bovine') and describes a sample taken from it (e.g., '...milk metagenome', '...gut', '...oral metagenome'), your output **MUST BE** the scientific name of that host organism. **Completely ignore the specific sample type** (milk, oral, skin, feces, etc.) in the output value. The value must *only* be the scientific name.
                2.  **The Null Value Rule:** For inputs like 'not applicable', 'not collected', 'N/A', or 'missing', the value **MUST** be the JSON literal `null`.
                3.  **The Consolidation Rule:** Consolidate all variations, common names, and strains to the main scientific name (e.g., 'C57 mice', 'mice1', 'C57BL/6' must all become *Mus musculus*).
                4.  **The Environmental Rule:** For environmental samples without a clear host (e.g., 'soil metagenome', 'aquatic metagenome'), use the original string as the value.

                ## Crucial Examples (Follow this logic precisely):
                -   `"human milk metagenome": "Homo sapiens"`
                -   `"human oral metagenome": "Homo sapiens"`
                -   `"human reproductive system metagenome": "Homo sapiens"`
                -   `"bovine metagenome": "Bos taurus"`
                -   `"Bradyrhizobium sp. W001": "Bradyrhizobium"`
                -   `"rat gut metagenome": "Rattus norvegicus"`
                -   `"human-associated": "Homo sapiens"`
                -   `"not collected": null`
                -   `"C57BL/6": "Mus musculus"`
                -   `"Rhizosphere rep 1": "soil metagenome"`
                -   `"soil metagenome": "soil metagenome"`

                ## Output Format:
                Your response must be ONLY a single, valid JSON object that maps each original name from the input list to its corrected, standardized version. Do not include any other text, explanations, or markdown formatting.

                Here is the list of hosts:\n"""
                f"{json.dumps(unique_hosts)}"
        )

        try:
            response_text = self._call_gemini_with_retries(prompt)
            # Clean the response to ensure it's valid JSON
            clean_json_text = response_text.strip().lstrip("```json").rstrip("```")
            normalization_map = json.loads(clean_json_text)
            return normalization_map
        except json.JSONDecodeError as e:
            print(f"  > LLM returned invalid JSON, cannot create normalization map. Error: {e}")
            return {}
        except Exception as e:
            print(f"  > An unexpected error occurred during LLM normalization: {e}")
            return {}

# --- API Request Helper ---
def make_request_with_retries(method, url, data=None, retries=3):

    for attempt in range(retries):
        try:
            if method.upper() == 'POST':
                response = requests.post(url, data=data)
            else:
                response = requests.get(url)
            if response.status_code in [429, 500, 502, 503, 504]:
                wait_time = 5 * (attempt + 1)
                print(f"  > WARNING: Received status {response.status_code}. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
                continue
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            print(f"  > ERROR: Request failed on attempt {attempt + 1}: {e}")
            if attempt < retries - 1: time.sleep(5 * (attempt + 1))
            else: return None
    return None

def get_cngb_details(accession_code: str):
    """
    Fetches project metadata from CNGBdb using the FTP protocol. This version
    derives the experiment count from the metadata file itself for better accuracy.
    """
    print(f"Fetching CNGBdb data for {accession_code}...")
    details = {'AccessionCode': accession_code}
    ftp = None

    try:
        # --- Connect to the FTP server and dynamically find files ---
        ftp_host = "ftp2.cngb.org"
        ftp_path = f"/pub/CNSA/data7/public_info/{accession_code}/"

        print(f"  > Connecting to FTP server: {ftp_host}")
        ftp = FTP(ftp_host)
        ftp.login()
        ftp.cwd(ftp_path)

        filenames = ftp.nlst()
        print(f"  > Found {len(filenames)} files in directory.")

        experiment_file_name = None
        sample_file_name = None

        for f in filenames:
            if 'experiment' in f.lower() and f.endswith('.tsv'):
                experiment_file_name = f
            if 'sample' in f.lower() and f.endswith('.tsv'):
                sample_file_name = f

        if not experiment_file_name or not sample_file_name:
            print(f"  > ERROR: Could not find both experiment and sample .tsv files in the directory.")
            return details

        # --- Download files into memory and read with pandas ---
        print(f"  > Found experiment file: {experiment_file_name}")
        exp_bio = io.BytesIO()
        ftp.retrbinary(f'RETR {experiment_file_name}', exp_bio.write)
        exp_bio.seek(0)
        exp_df = pd.read_csv(exp_bio, sep='\t', on_bad_lines='skip')

        print(f"  > Found sample file: {sample_file_name}")
        sample_bio = io.BytesIO()
        ftp.retrbinary(f'RETR {sample_file_name}', sample_bio.write)
        sample_bio.seek(0)
        sample_df = pd.read_csv(sample_bio, sep='\t', on_bad_lines='skip')

        # --- Get the experiment count directly from the DataFrame length ---
        details['Number of Experiments'] = len(exp_df)

        # Extract and format the data
        details['All Experiments IDs'] = '; '.join(sorted(list(set(exp_df['experiment_accession'].dropna()))))
        details['SRA Name'] = '; '.join(sorted(list(set(exp_df['sample_name'].dropna()))))
        details['Library Name'] = '; '.join(sorted(list(set(exp_df['library_name'].dropna()))))

        combined_instruments = (exp_df['platform'].fillna('') + ' ' + exp_df['instrument_model'].fillna('')).str.strip()
        details['Instrument Name'] = '; '.join(sorted(list(set(combined_instruments.dropna()))))

        unique_biosamples = sorted(list(set(sample_df['sample_accession'].dropna())))
        details['BioSample Accession'] = '; '.join(unique_biosamples)
        details['Sample Title'] = '; '.join(sorted(list(set(sample_df['sample_title'].dropna()))))
        details['SRA Host'] = '; '.join(sorted(list(set(sample_df['host'].dropna()))))

        # Extract all BioSample attributes
        attr_df = sample_df.drop(columns=['sample_accession', 'sample_name', 'sample_title'], errors='ignore')
        details['All Attribute Keys'] = '; '.join(sorted(attr_df.columns))

        all_attributes = defaultdict(list)
        for col in attr_df.columns:
            all_attributes[col].extend(attr_df[col].dropna().unique().tolist())

        details['BioSampleAttributes'] = {key: sorted(list(set(values))) for key, values in all_attributes.items()}

        print(f"  > Successfully processed {accession_code}.")
        return details

    except Exception as e:
        print(f"  > An unexpected critical error occurred for {accession_code}: {e}")
        return { 'AccessionCode': accession_code, 'Error': str(e) }
    finally:
        # Ensure the FTP connection is always closed
        if ftp:
            ftp.quit()
            print("  > FTP connection closed.")

def get_ngdc_details(accession_code: str):
    """
    Fetches and parses project metadata from the China National Genomics
    Data Center (NGDC) for a given PRJCA accession.
    """
    print(f"Fetching NGDC data for {accession_code}...")
    details = {'AccessionCode': accession_code}

    try:
        # Get Project Title from the BioProject API
        project_api_url = f"https://ngdc.cncb.ac.cn/gwh/api/public/bioProject/{accession_code}"
        project_response = make_request_with_retries('GET', project_api_url)
        if not project_response: return details

        project_data = project_response.json()
        details['SRA Name'] = project_data.get('title', 'N/A')

        # 2. Scrape the HTML browse page to get the sample count and all sample accessions
        browse_url = f"https://ngdc.cncb.ac.cn/bioproject/browse/{accession_code}"
        print(f"  > Scraping for sample list: {browse_url}")
        browse_response = make_request_with_retries('GET', browse_url)
        if not browse_response: return details

        html_content = browse_response.text
        # Extract sample count
        count_match = re.search(r"BioSample\s*\((\d+)\)", html_content)
        details['Number of Experiments'] = int(count_match.group(1)) if count_match else 0

        # Extract all sample accessions (e.g., SAMC#######)
        sample_ids = re.findall(r'/browse/(SAMC\d+)', html_content)
        unique_biosamples = sorted(list(set(sample_ids)))
        details['BioSample Accession'] = '; '.join(unique_biosamples)
        print(f"  > Found {len(unique_biosamples)} unique BioSample accessions.")

        if not unique_biosamples:
            return details

        # For each BioSample, query its API to get detailed metadata
        all_exp_ids, all_lib_names, all_inst_names, all_sample_titles, all_sra_hosts = [], [], [], [], []
        all_attributes = defaultdict(list)

        for i, sample_id in enumerate(unique_biosamples):
            print(f"\r  > Processing sample {i+1}/{len(unique_biosamples)}: {sample_id}", end="")
            sample_api_url = f"https://ngdc.cncb.ac.cn/gwh/api/public/bioSample/{sample_id}"
            sample_response = make_request_with_retries('GET', sample_api_url)
            if not sample_response: continue

            sample_data = sample_response.json()

            # Collect sample-level info
            if (st := sample_data.get('name')): all_sample_titles.append(st)
            if (host := sample_data.get('sampleAttribute',{}).get('hostTaxon', {}).get('name')): all_sra_hosts.append(host)

            # Collect attributes

            attributes_dict = sample_data.get('sampleAttribute', {})
            for attrName, attrValue in attributes_dict.items():
                if attrValue is not None:
                    all_attributes[attrName]=attrValue

            # Collect experiment-level info from within the sample data
            # for exp in sample_data.get('experiments', []):
            #     if (eid := exp.get('experimentAccession')): all_exp_ids.append(eid)
            #     if (ln := exp.get('libraryName')): all_lib_names.append(ln)
            #     # NGDC provides 'platform', which we'll use for Instrument Name
            #     if (im := exp.get('platform')): all_inst_names.append(im)

            # time.sleep(0.1) # Small delay to be polite to the server
            # break
        print("\n  > Sample processing complete.")

        # Aggregate and format all collected data
        details['All Experiments IDs'] = ''
        details['Library Name'] = ''
        details['Instrument Name'] = ''
        details['Sample Title'] = '; '.join(sorted(list(set(all_sample_titles))))
        details['SRA Host'] = '; '.join(sorted(list(set(all_sra_hosts))))

        details['All Attribute Keys'] = '; '.join(sorted(all_attributes.keys()))
        details['BioSampleAttributes'] = '; '.join(sorted(set([f"{key}: {values}" for key, values in all_attributes.items()])))

        print(f"  > Successfully processed {accession_code}.")
        return details

    except Exception as e:
        print(f"  > An unexpected critical error occurred for {accession_code}: {e}")
        return { 'AccessionCode': accession_code, 'Error': str(e) }

# --- Main Data Fetching Function (Corrected Signature) ---
def get_sra_and_biosample_details(accession_code: str):
    """
    Main router function. Checks the accession prefix and calls the appropriate
    function for either NCBI or CNGBdb.
    """
    # Define valid prefixes for both repositories
    NCBI_PREFIXES = ('PRJDB','PRJNA', 'PRJEB', 'SRP', 'E-MTAB', 'ERA', 'ERP', 'DRP', 'DRA', 'DRX', 'SAMN', 'SRX', 'SRR', 'PHS')
    CNGB_PREFIXES = ('CNP',)
    NGDC_PREFIXES = ('PRJCA',)

    # --- ROUTER LOGIC ---
    if isinstance(accession_code, str):
        upper_accession = accession_code.upper()
        if upper_accession.startswith(NCBI_PREFIXES):
            return get_ncbi_details(accession_code)
        elif upper_accession.startswith(CNGB_PREFIXES):
            return get_cngb_details(accession_code)
        elif upper_accession.startswith(NGDC_PREFIXES):
            # Route to the new NGDC function
            return get_ngdc_details(accession_code)

    # If it's not a supported format, skip it
    print(f"Skipping invalid or unsupported accession format: {accession_code}")
    return {}


def get_ncbi_details(accession_code: str):
    """
    Main function to query NCBI. This version only fetches raw data and has no
    dependency on the LLM normalizer.
    """

    main_accession = accession_code.split(',')[0].strip()
    details = { 'AccessionCode': accession_code }
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"

    try:
        # Step 1: Get experiment count and all internal IDs
        print(f"Fetching SRA data for {main_accession}...")
        esearch_url = f"{base_url}esearch.fcgi?db=sra&term={main_accession}"
        time.sleep(0.4)
        response = make_request_with_retries('GET', esearch_url)
        if not response: return details
        root = ET.fromstring(response.content)
        count = int(root.find('.//Count').text)
        details['Number of Experiments'] = count
        print(f"  > Experiments found {count}")
        if count == 0:
            print(f"  > No experiments found for {main_accession}.")
            return details

        esearch_all_ids_url = f"{base_url}esearch.fcgi?db=sra&term={main_accession}&retmax={count}"
        all_ids_response = make_request_with_retries('GET', esearch_all_ids_url)
        if not all_ids_response: return details
        all_ids_list = [elem.text for elem in ET.fromstring(all_ids_response.content).findall('.//Id')]

        # Step 2: Fetch SRA metadata in batches
        all_sra_data = []
        chunk_size = 200
        for i in range(0, len(all_ids_list), chunk_size):
            chunk = all_ids_list[i:i + chunk_size]
            print(f"  > Fetching SRA metadata for batch {i//chunk_size + 1}...")
            post_data = {'db': 'sra', 'id': ','.join(chunk), 'retmode': 'xml'}
            efetch_response = make_request_with_retries('POST', f"{base_url}efetch.fcgi", data=post_data)
            if efetch_response: all_sra_data.append(ET.fromstring(efetch_response.content))

        # Step 3: Parse aggregated SRA data
        all_exp_ids, all_titles, all_lib_names, all_inst_names, all_sample_titles, all_biosample_accs = [], [], [], [], [], []
        for sra_root in all_sra_data:
            for exp in sra_root.findall('.//EXPERIMENT'):
                if (eid := exp.find('./IDENTIFIERS/PRIMARY_ID')) is not None and eid.text: all_exp_ids.append(eid.text.strip())
                if (t := exp.find('./TITLE')) is not None and t.text: all_titles.append(t.text.strip())
                if (ln := exp.find('.//LIBRARY_NAME')) is not None and ln.text: all_lib_names.append(ln.text.strip())
                if (im := exp.find('.//INSTRUMENT_MODEL')) is not None and im.text: all_inst_names.append(im.text.strip())
            for sample in sra_root.findall('.//SAMPLE'):
                if (st := sample.find('./TITLE')) is not None and st.text: all_sample_titles.append(st.text.strip())
                for ext_id in sample.findall('.//EXTERNAL_ID'):
                    if ext_id.get('namespace') == 'BioSample' and ext_id.text: all_biosample_accs.append(ext_id.text.strip())

        details['All Experiments IDs'] = '; '.join(sorted(list(set(all_exp_ids))))
        details['SRA Name'] = '; '.join(sorted(list(set(all_titles))))
        details['Library Name'] = '; '.join(sorted(list(set(all_lib_names))))
        details['Instrument Name'] = '; '.join(sorted(list(set(all_inst_names))))
        details['Sample Title'] = '; '.join(sorted(list(set(all_sample_titles))))
        unique_biosamples = sorted(list(set(all_biosample_accs)))
        details['BioSample Accession'] = '; '.join(unique_biosamples)

        # Step 4: Fetch BioSample attributes and store them raw
        if unique_biosamples:
            print(f"  > Found {len(unique_biosamples)} unique BioSamples. Fetching all raw attributes...")
            all_attributes = defaultdict(list)
            for i in range(0, len(unique_biosamples), chunk_size):
                chunk = unique_biosamples[i:i + chunk_size]
                post_data = {'db': 'biosample', 'id': ','.join(chunk), 'retmode': 'xml'}
                biosample_response = make_request_with_retries('POST', f"{base_url}efetch.fcgi", data=post_data)
                if not biosample_response: continue
                for biosample in ET.fromstring(biosample_response.content).findall('BioSample'):
                    for attr in biosample.findall('.//Attribute'):
                        if (attr_name := attr.get('attribute_name')) and attr.text:
                            all_attributes[attr_name].append(attr.text.strip())

            details['SRA Host'] = '; '.join(sorted(list(set(all_attributes.get('host', [])))))
            details['All Attribute Keys'] = '; '.join(sorted(all_attributes.keys()))
            details['BioSampleAttributes'] = {key: sorted(list(set(values))) for key, values in all_attributes.items()}

        print(f"  > Successfully processed {main_accession}.")
        return details

    except Exception as e:
        print(f"  > An unexpected critical error occurred for {main_accession}: {e}")
        return { 'AccessionCode': accession_code, 'Error': str(e) }


### Step 3

In [ ]:
# ==============================================================================
# STEP 3: LOAD DATA FROM GOOGLE SHEET
# ==============================================================================

# --- Load the full, current dataset from Google Sheets ---
print("--- Loading data from Google Sheet ---")

# --- Open the spreadsheet and the specific sheet by name ---
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1tBjpV_GoXIjx4_3o73Qml0h-BzFBZTD5bc3QHx1l1_4'
spreadsheet = gc.open_by_url(spreadsheet_url)
worksheet = spreadsheet.worksheet("Main Data Sheet")

# --- Manually read the data to handle the header on the second row ---
all_values = worksheet.get_all_values()

# The header is the second row (index 1), and the data starts from the third row (index 2)
header = all_values[1]
data_rows = all_values[2:2349]

# --- Convert the lists to a pandas DataFrame --
main_data_sheet = pd.DataFrame(data_rows, columns=header)

# --- Reset the index to ensure it is unique and clean ---
# main_data_sheet.reset_index(drop=True, inplace=True)

# Clean up the DataFrame: remove rows where 'AccessionCode' is empty
# main_data_sheet = main_data_sheet[main_data_sheet['AccessionCode'] != ''].copy()
print(f"Loaded {len(main_data_sheet)} total records from Google Sheet.")

# --- Identify new records by checking against a saved list of processed accessions ---
new_rows_to_process = pd.DataFrame()

# Filter for rows that are not in the processed list
new_rows_to_process = main_data_sheet[main_data_sheet['Number of Experiments']==""]

print(f"Found {len(new_rows_to_process)} new records to process.")

print("\n--- Successfully loaded data from Google Sheet ---")

# Display the first few rows of the loaded data
DataTable(new_rows_to_process.head())

--- Loading data from Google Sheet ---
Loaded 2347 total records from Google Sheet.
Found 1087 new records to process.

--- Successfully loaded data from Google Sheet ---


,Author1Author2name (LastName1LastName2),Has someone done this study?,YourName,Host,Environment,StudyLink,StudyTitle,Year,Country,Disease,...,attr_env_medium,attr_isolate,attr_sex,Calculated Country,disease_name,classification_confidence,classification_reasoning,icd11_code,icd11_description,icd11_search_confidence
13,UsykZolnik2020,you're good!,EmilySong,human,Vaginal,https://journals.plos.org/plospathogens/articl...,Cervicovaginal microbiome and natural history ...,2020,United States,,...,,,,,Cervical intraepithelial neoplasia,high,The study explicitly investigates the role of ...,GA15.7,Low grade squamous intraepithelial lesion of c...,
25,ChunDo2021,you're good!,HarrisonGu,human,Nasal,https://www.jacionline.org/article/S0091-6749(...,"The nasal microbiome, nasal transcriptome, and...",2021,United States,,...,,,,,Pet allergies,high,The title and abstract explicitly state that t...,,,
28,BaedeBarray2022,you're good!,HarrisonGu,human,Nasal,https://pmc.ncbi.nlm.nih.gov/articles/PMC9671894/,Nasal microbiome disruption and recovery after...,2022,France,,...,,,,,Not processed,,Could not retrieve article details from PubMed.,Not processed,,
29,FazlollahiLee2018,you're good!,HarrisonGu,human,Nasal,https://pmc.ncbi.nlm.nih.gov/articles/PMC6123291/,The Nasal Microbiome in Asthma,2018,United States,,...,,,,,Asthma,high,"The title, abstract, and methods section all e...",CA23,Asthma,
30,IrizarChun2023,you're good!,HarrisonGu,human,Nasal,https://pmc.ncbi.nlm.nih.gov/articles/PMC9643606/,Examination of host genetic effects on nasal m...,2022,United States,,...,,,,,Asthma,high,The abstract explicitly states that 'Genetic p...,CA23,Asthma,


### Step 4

In [ ]:
# ==============================================================================
# STEP 4: PROCESS EACH ACCESSION CODE (MAIN FETCHING LOOP)
# ==============================================================================
new_results = []
print(f"\n--- Starting NCBI/CNGB Data Extraction for {len(new_rows_to_process)} new records (Pass 1) ---")

valid_new_rows = new_rows_to_process[new_rows_to_process.get("Has someone done this study?") != "Someone else did this study!"].copy()
print(f"Skipping {len(new_rows_to_process) - len(valid_new_rows)} records marked as done by someone else.")

if not valid_new_rows.empty:
    for index, row in valid_new_rows.iterrows():
        # The 'index' from iterrows() is the original index from main_data_sheet
        accession = row['AccessionCode'].strip()
        sra_details = get_sra_and_biosample_details(accession)

        if sra_details:
            # IMPORTANT: Store the original index with the results
            sra_details['original_index'] = index
            new_results.append(sra_details)

        print(f"  > Processed {index} accessions.")

print("\n--- Raw Data Extraction Complete ---")


--- Starting NCBI/CNGB Data Extraction for 1087 new records (Pass 1) ---
Skipping 2 records marked as done by someone else.
Skipping invalid or unsupported accession format: n/a
  > Processed 13 accessions.
Skipping invalid or unsupported accession format: n/a
  > Processed 25 accessions.
Skipping invalid or unsupported accession format: https://zenodo.org/records/6382657
  > Processed 28 accessions.
Skipping invalid or unsupported accession format: n/a
  > Processed 29 accessions.
Skipping invalid or unsupported accession format: n/a
  > Processed 30 accessions.
Skipping invalid or unsupported accession format: n/a
  > Processed 34 accessions.
Skipping invalid or unsupported accession format: n/a
  > Processed 35 accessions.
Skipping invalid or unsupported accession format: n/a
  > Processed 40 accessions.
Skipping invalid or unsupported accession format: n/a
  > Processed 41 accessions.
Skipping invalid or unsupported accession format: n/a
  > Processed 43 accessions.
Skipping inval

### Step 5

In [ ]:
# ==============================================================================
# STEP 5: POST-PROCESSING AND GLOBAL NORMALIZATION (PASS 2)
# ==============================================================================
print("\n--- Starting Global Post-Processing (Pass 2) ---")

if not new_results:
    print("No new data was processed. Final output will match the original sheet.")
    # If there are no new results, we can skip straight to the end
    results_df = pd.DataFrame() # Create an empty DataFrame
else:
    print("\n--- Starting Global Post-Processing on new data (Pass 2) ---")
    results_df = pd.DataFrame(new_results)

    # --- Part A: Determine Top 10 Attribute Keys ---
    if 'BioSampleAttributes' in results_df.columns:
        print("  > Aggregating BioSample attributes across all new projects...")
        global_key_counts = defaultdict(int)
        # Ensure the column is treated as dictionaries
        results_df['BioSampleAttributes'] = results_df['BioSampleAttributes'].apply(lambda x: x if isinstance(x, dict) else {})

        for attr_dict in results_df['BioSampleAttributes']:
            for key in attr_dict.keys():
                global_key_counts[key] += 1

        # Get existing "attr_" columns from the original sheet for the new rows
        existing_attr_columns = [col for col in new_rows_to_process.columns if col.startswith('attr_')]
        existing_attr_keys = [col.replace('attr_', '') for col in existing_attr_columns]


        # Determine which keys to process (either top 10 new or all existing)
        if existing_attr_columns:
            print("  > Existing 'attr_' columns found. Filling these columns.")
            keys_to_process = existing_attr_keys
        else:
            print("  > No existing 'attr_' columns found. Determining top 10 from new data.")
            keys_to_process = sorted(global_key_counts, key=global_key_counts.get, reverse=True)[:10]

        print(f"  > Keys to process: {keys_to_process}")

        for key in keys_to_process:
            col_name = f"attr_{key.replace(' ', '_').lower()}"
            # If the column exists, update it; otherwise, create it
            if col_name in results_df.columns and existing_attr_columns:
                 print(f"  > Filling existing column: {col_name}")
                 results_df[col_name] = results_df['BioSampleAttributes'].apply(
                    lambda attr_dict: '; '.join(attr_dict.get(key, []))
                 )
            else:
                print(f"  > Creating new column: {col_name}")
                results_df[col_name] = results_df['BioSampleAttributes'].apply(
                    lambda attr_dict: '; '.join(attr_dict.get(key, []))
                )
        results_df = results_df.drop(columns=['BioSampleAttributes'])

    # --- Part B: Normalize Host Column Using LLM ---
    if 'SRA Host' in results_df.columns:
        print("  > Normalizing 'SRA Host' column with LLM...")
        llm = LLMNormalizer()
        if llm.model:
            all_raw_hosts = set()
            for host_string in results_df['SRA Host'].dropna().astype(str):
                for host in host_string.split('; '):
                    if host: all_raw_hosts.add(host)
            final_host_normalization_map = llm.create_host_normalization_map(list(all_raw_hosts))
            if final_host_normalization_map:
                print(f"  > Created normalization map for {len(final_host_normalization_map)} unique host values.")
                def apply_normalization(host_string):
                    if not isinstance(host_string, str) or not host_string: return ''
                    hosts = host_string.split('; ')
                    normalized_values = {final_host_normalization_map.get(h.strip(), h.strip()) for h in hosts}
                    final_hosts = sorted([host for host in normalized_values if host is not None])
                    return '; '.join(final_hosts)
                results_df['SRA Host'] = results_df['SRA Host'].apply(apply_normalization)
                print("  > 'SRA Host' column normalized.")

# --- Part C (in this step): Set the index for joining ---
# This is critical for the next step.
if not results_df.empty:
    results_df.set_index('original_index', inplace=True)



--- Starting Global Post-Processing (Pass 2) ---

--- Starting Global Post-Processing on new data (Pass 2) ---
  > Aggregating BioSample attributes across all new projects...
  > Existing 'attr_' columns found. Filling these columns.
  > Keys to process: ['geo_loc_name', 'lat_lon', 'host', 'isolation_source', 'env_broad_scale', 'env_local_scale', 'env_medium', 'isolate', 'sex']
  > Creating new column: attr_geo_loc_name
  > Creating new column: attr_lat_lon
  > Creating new column: attr_host
  > Creating new column: attr_isolation_source
  > Creating new column: attr_env_broad_scale
  > Creating new column: attr_env_local_scale
  > Creating new column: attr_env_medium
  > Creating new column: attr_isolate
  > Creating new column: attr_sex
  > Normalizing 'SRA Host' column with LLM...
LLM Normalizer initialized successfully.
  > Created normalization map for 515 unique host values.
  > 'SRA Host' column normalized.


In [ ]:
# ==============================================================================
# STEP 6: CREATE FINAL MERGED OUTPUT
# ==============================================================================
print("\n--- Coalescing new data into the main sheet ---")

# Start with a direct copy of the original data
final_output_df = main_data_sheet.copy()

if not results_df.empty:
    # The `results_df` index was set to 'original_index' in Step 5.
    # The .update() method aligns on the index and overwrites existing
    # values in `final_output_df` with new data from `results_df`.
    # This acts as a "coalesce," filling in the processed data perfectly.
    # It only affects rows and columns that are present in `results_df`.
    final_output_df.update(results_df)

# Final cleanup of any remaining nulls or placeholder text
final_output_df.fillna('', inplace=True)
final_output_df.replace(['N/A', 'null'], '', inplace=True)

print(f"Final DataFrame created with {len(final_output_df)} rows.")
display(final_output_df.head())

# Save the final file
final_output_df.to_csv('final_output.csv', index=False)
print("Saved final data to 'final_output.csv'")


--- Coalescing new data into the main sheet ---
Final DataFrame created with 2347 rows.


,Author1Author2name (LastName1LastName2),Has someone done this study?,YourName,Host,Environment,StudyLink,StudyTitle,Year,Country,Disease,...,attr_env_medium,attr_isolate,attr_sex,Calculated Country,disease_name,classification_confidence,classification_reasoning,icd11_code,icd11_description,icd11_search_confidence
0,ZhangLiu2020,you're good!,SamuelDegregori,human,Oral,https://pmc.ncbi.nlm.nih.gov/articles/pmid/320...,The Oral Microbiota May Have Influence on Oral...,2020,China,,...,,,,China,Oral cancer,high,"The title, abstract, and methods section expli...",2B6E,Malignant neoplasms of other or ill-defined si...,high
1,Lev-SagieGoldman-Wohl2019,you're good!,EmilySong,human,Vaginal,https://www.nature.com/articles/s41591-019-060...,Vaginal microbiome transplantation in women wi...,2019,Israel,,...,,,,,Bacterial vaginosis,high,The title and abstract explicitly state that t...,,,
2,BaldiBraat2024,you're good!,SolanaCallaway,human,Gut,https://pubmed.ncbi.nlm.nih.gov/39367018/,Effects of iron supplements and iron-containin...,2024,Bangladesh,,...,,,,Bangladesh,Anemia,high,The abstract explicitly states that anemia is ...,,Anaemias or other erythrocyte disorders,
3,HosangCanals2022,you're good!,VictoriaXu,human,Pulmonary,https://www.nature.com/articles/s41586-022-044...,The lung microbiome regulates brain autoimmunity,2022,Germany,,...,,,,Germany,Multiple sclerosis,high,The abstract explicitly states that the study ...,8A40,Multiple sclerosis,
4,XiaoCai2022,you're good!,AnjaliVinodh,human,Pulmonary,https://pmc.ncbi.nlm.nih.gov/articles/PMC88654...,Insights into the Unique Lung Microbiota Profi...,2022,China,,...,BAL; Swab,,,China,Multiple sclerosis,high,The abstract explicitly states that the resear...,8A40,Multiple sclerosis,high


Saved final data to 'final_output.csv'


In [ ]:
# ==============================================================================
# ONE-TIME SCRIPT TO MERGE EXISTING PROCESSED FILE WITH MAIN SHEET
# ==============================================================================
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default

print("--- Starting one-time merge process ---")

# --- Authenticate and Load Google Sheet ---
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

print("Loading data from Google Sheet...")
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1tBjpV_GoXIjx4_3o73Qml0h-BzFBZTD5bc3QHx1l1_4'
spreadsheet = gc.open_by_url(spreadsheet_url)
worksheet = spreadsheet.worksheet("Main Data Sheet")
all_values = worksheet.get_all_values()
header = all_values[1]
data_rows = all_values[2:]
main_data_sheet = pd.DataFrame(data_rows, columns=header)
print(f"Loaded {len(main_data_sheet)} records from Google Sheet.")

# --- Load and Prepare Processed Data ---
PROCESSED_FILE = 'all_rows.csv'
try:
    print(f"Loading data from {PROCESSED_FILE}...")
    processed_df = pd.read_csv(PROCESSED_FILE)

    # --- Define the columns you want to merge from the processed file ---
    columns_to_merge = [
        'AccessionCode',
        'Number of Experiments',
        'All Experiments IDs',
        'SRA Name',
        'Library Name',
        'Instrument Name',
        'Sample Title',
        'BioSample Accession',
        'SRA Host',
        'All Attribute Keys',
        'attr_geo_loc_name',
        'attr_lat_lon',
        'attr_host',
        'attr_isolation_source',
        'attr_env_broad_scale',
        'attr_env_local_scale',
        'attr_env_medium',
        'attr_isolate',
        'attr_sex'
    ]

    # Ensure all desired columns exist in the DataFrame, adding them if they don't
    for col in columns_to_merge:
        if col not in processed_df.columns:
            processed_df[col] = '' # Add missing columns as blank

    # Select only the columns we want to merge
    processed_df_to_merge = processed_df[columns_to_merge]

    # CRITICAL: Drop duplicates based on 'AccessionCode' to ensure a clean merge
    processed_df_to_merge.drop_duplicates(subset=['AccessionCode'], inplace=True)
    print(f"Prepared {len(processed_df_to_merge)} unique processed records for merging.")

    # --- Perform the Merge ---
    # Use a left merge to keep all rows from the original sheet
    print("Merging dataframes based on 'AccessionCode'...")
    final_merged_df = pd.merge(main_data_sheet, processed_df_to_merge, on='AccessionCode', how='left')
    print(f"New Lengtgh {len(final_merged_df)}")
    # --- Final Cleanup and Save ---
    final_merged_df.fillna('', inplace=True)
    if len(final_merged_df) == len(main_data_sheet):
      final_merged_df.to_csv('one_time_merge_result.csv', index=False)
    else:
      print("Issue with merge")

    print("\n--- Merge Complete ---")
    print("Saved final merged data to 'one_time_merge_result.csv'")
    # display(final_merged_df.head())

except FileNotFoundError:
    print(f"ERROR: The file '{PROCESSED_FILE}' was not found. Cannot perform merge.")


--- Starting one-time merge process ---
Loading data from Google Sheet...
Loaded 2251 records from Google Sheet.
Loading data from all_rows.csv...


/tmp/ipython-input-41-3311301256.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df_to_merge.drop_duplicates(subset=['AccessionCode'], inplace=True)
/tmp/ipython-input-41-3311301256.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  final_merged_df.fillna('', inplace=True)


Prepared 1130 unique processed records for merging.
Merging dataframes based on 'AccessionCode'...
New Lengtgh 2251

--- Merge Complete ---
Saved final merged data to 'one_time_merge_result.csv'
